# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 2: Detección de SPAM

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline

### Explorar Datos

In [14]:
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.drop(["Unnamed: 0"], axis=1, inplace=True)
df1.sample(5)

,Body,Label
4042,"URL: http://www.newsisfree.com/click/-2,841883...",0
1331,\n Real \n\t Drugs - Viagra and Phentrimine\nO...,1
1544,.ukqdrblkougooeoffnxsmbowei\n,1
711,To remove see below.I work with a company that...,1
4641,"Hi guys,\n\tI'm looking for help on this one. ...",0


In [13]:
df2 = pd.read_csv('enronSpamSubset.csv')
df2.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)
df2.sample(5)

,Body,Label
7344,Subject: risk assessment\n this is to confirm ...,0
7161,"Subject: vince ,\n the book "" power economics ...",0
9863,Subject: document\n change the name to\n get y...,0
8071,Subject: tractebel signed vitro\n ena entered ...,0
6633,Subject: update : the meeting will be in 49 cl...,0


### Merge de los df's

In [4]:
dfNew = df1.merge(df2, how="right")
dfNew.sample(5)

,Body,Label
9093,Subject: re : hpl meter # 986563 texaco fee c ...,0
3572,"Subject: ouur medz\n hello , welcome to pharmo...",1
3350,"Subject: perfect logo charset = koi 8 - r "" >\...",1
8081,Subject: re :\n fine with me\n - - - - - origi...,0
1437,Subject: 77 % off for all new software . outra...,1


### Pre Procesamiento

In [5]:
#Conversión de Mayusculas a Minusculas
def Conv_Up_to_Low(df):
    sentences = []
    string = ""
    for sentence in df.Body:
        string = str(sentence)
        sentences.append(string.lower())
        
    df["Body"] = sentences
    
    return df

#Acentos
import unicodedata
def accents(df):
    sentences = []
    for sentence in df.Body:
        sentences.append(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore').decode('utf-8','ignore'))
    
    df["Body"] = sentences
    
    return df

#Caracteres especiales, números y simbolos
import re
def removeSpecialCharacters(df):
    patron = r'[^a-zA-Z0-9\s]'
    sentences = []
    for sentence in df.Body:
        if patron in sentence:
            sentences.append(re.sub(patron,'', sentence))
        else:
            sentences.append(re.sub(r'[^a-zA-Z\s]', '', sentence))
    
    df["Body"] = sentences
    return df
    
#Contracciones en el texto
import contractions
def contrac(df):
    list(contractions.contractions_dict.items())[:10]
    sentences = []
    for sentence in df.Body:
        sentences.append(contractions.fix(sentence))
        
    df["Body"] = sentences
    return df

#Stop Words
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def Stop_word(df):
    sentences = []
    for sentence in df.Body:
        tokens = nltk.word_tokenize(sentence.lower())
        stop_words_spanish = nltk.corpus.stopwords.words('spanish')
        tokens_filtrados = [token for token in tokens if token not in stop_words_spanish]
        stop_words_english = nltk.corpus.stopwords.words('english')
        tokens_english = nltk.word_tokenize(contractions.fix(sentence))
        tokens_filtrados_english = [token for token in tokens_english if token not in stop_words_english]
        sentences.append(tokens_filtrados_english)
        
    df["Body"] = sentences
    return df
    

[nltk_data] Downloading package punkt to C:\Users\Javier
[nltk_data]     Cotto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Javier
[nltk_data]     Cotto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
dfNew = Conv_Up_to_Low(dfNew)
dfNew = accents(dfNew)
dfNew = removeSpecialCharacters(dfNew)
dfNew = contrac(dfNew)
dfNew = Stop_word(dfNew)

In [7]:
dfNew

,Body,Label
0,"[subject, stock, promo, mover, cwtd, urgent, i...",1
1,"[subject, listed, major, search, engines, subm...",1
2,"[subject, important, information, thu, jun, su...",1
3,"[subject, utf, q, bask, life, utf, q, individu...",1
4,"[subject, bidstogo, places, go, things, hello,...",1
...,...,...
9995,"[subject, monday, nd, oct, louise, half, hour,...",0
9996,"[subject, missing, bloomberg, deals, stephanie...",0
9997,"[subject, eops, salary, survey, questionnaire,...",0
9998,"[subject, q, comparison, hi, louise, compariso...",0


In [54]:
norm_corpus = []
palabra = ""
for lista in dfNew.Body:
    empty_str = ""
    for word in lista:
        if word != "subject":
            empty_str += word + " "
        
    norm_corpus.append(empty_str)
    
norm_corpus
#norm_corpus = np.array(norm_corpus)
#norm_corpus

['stock promo mover cwtd urgent investor trading alert weekly stock pick china world trade corp ticker cwtd breaking news china world trade corp enters agreement acquire majority stake ceo clubs china limited ceo clubs tianhe guangzhou china market wire apr china world trade corp otc bb cwtd ob news announced today entered agreement acquired majority stake ceo clubs china limited ceo clubs hong kong corporation authorized chapter operate ceo clubs trademarks greater china region including prc hong kong taiwan china world trade corp symbol cwtd price shares million market capitalization million significant revenue growth average pe industry x rating strong buy days trading target day trading target cwtd play month stock pick simple reasons one would china world trade corp china world trade corp announced today entered agreement acquired majority stake ceo clubs china limited ceo clubs hong kong corporation authorized chapter operate ceo clubs trademarks greater china region including pr

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.1, max_df=0.8)
# Min_df y Max_df nos sirven para controlar el porcentaje mínimo y máximo de apariciones de un token
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [58]:
# Palabras únicas del corpus
vocabulario = cv.get_feature_names()
# Mostrar el vector
pd.DataFrame(cv_matrix, columns=vocabulario)

C:\Users\Javier Cotto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,also,attached,best,business,call,cc,com,company,contact,could,...,take,thanks,time,today,us,use,want,work,would,www
0,0,0,0,4,0,0,0,6,0,2,...,0,0,2,2,0,1,0,1,1,0
1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,5,0,0,0,0,...,0,2,5,0,3,0,0,0,0,0
9997,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
